In [1]:
from KG.kg import ingest_Chunks, embed_text, create_nodes, create_relationship, create_vector_index
from KG.chunking import split_data_from_file
from KG.config import load_neo4j_graph
import json
graph = load_neo4j_graph()

#### Ingest main nodes and chunk data

In [2]:
file_names = ["Talleyrand", "Napoleon", "Battle_of_Waterloo"]

for name in file_names:
    #  Load JSON file
    file = f"Data/{name}.json"
    # Chunking
    chunks = split_data_from_file(file)
    # Assuming `file` is a path to your JSON file
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    if name == "Battle_of_Waterloo":
        create_nodes(graph=graph, data=data, node_label="Event", node_name=name)
    else:
        create_nodes(graph=graph, data=data, node_label="Person", node_name=name)
    # Ingest Chunks
    ingest_Chunks(graph=graph, chunks=chunks, node_name=name, node_label='Chunk')

['General Information', 'Career', 'Death']
Processing General Information from Data/Talleyrand.json
	Split into 12 chunks
Processing Career from Data/Talleyrand.json
	Split into 13 chunks
Processing Death from Data/Talleyrand.json
	Split into 1 chunks
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0000
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0001
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0002
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0003
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0004
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0005
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0006
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0007
Creating `:Chunk` node for chunk ID Talleyrand-General Information-chunk0008
Creating `:Chunk` node for chunk ID Talleyrand-General 

#### Add relationships

In [3]:
# Create relationship
rel_section_chunk = """ 
MATCH (s:Section), (c:Chunk)
WHERE s.type = c.source AND s.parent_name = c.node_name
MERGE (s)-[:HAS_CHUNK]->(c);

"""

rel_person_person = """
MATCH (p1:Person), (p2:Person)
WHERE id(p1) < id(p2)
MERGE (p1)-[:RELATED_TO]->(p2)
MERGE (p2)-[:RELATED_TO]->(p1);

"""

rel_person_event = """
MATCH (p:Person), (e:Event)
MERGE (p)-[:RELATED_TO]->(e)
MERGE (e)-[:RELATED_TO]->(p);

"""

rel_person_section = """
MATCH (p:Person), (s:Section)
WHERE p.name = s.parent_name
MERGE (p)-[:HAS_SECTION]->(s);

"""

rel_event_section = """
MATCH (e:Event), (s:Section)
WHERE e.name = s.parent_name
MERGE (e)-[:HAS_SECTION]->(s);

"""

queries = [rel_section_chunk, rel_person_person, rel_person_event, rel_person_section, rel_event_section]

for query in queries:
    create_relationship(graph=graph, query=query)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 7, offset: 38} for query: '\nMATCH (p1:Person), (p2:Person)\nWHERE id(p1) < id(p2)\nMERGE (p1)-[:RELATED_TO]->(p2)\nMERGE (p2)-[:RELATED_TO]->(p1);\n\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 16, offset: 47} for query: '\nMATCH (p1:Person), (p2:Person)\nWHERE id(p1) < id(p2)\nMERGE (p1)-[:RELATED_TO]->(p2)\nMERGE (p2)-[:RELATED_TO]->(p1);\n\n'


#### First ingest all data and then create index

In [6]:
# Create Vector Index
create_vector_index(graph=graph, index_name='Chunk')